In [1]:
import sys
import os
import scipy.io as sio
import cv2
import torch
import time
import numpy as np
import onnx
import onnxruntime
import MNN
import torch.nn.functional as F
import torch.nn as nn
from model.dbface_small import DBFace as dbface
from model.dbface_light import DBFace as dbface_light
from utils import preprocess
from model.losses import GIoULoss
from tqdm import tqdm

#### 构建模型

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
model = dbface_light(has_landmark=False, wide=24, has_ext=False, upmode="UCBA")
model.eval()

#### 配置输入

In [ ]:
mean = [0.408, 0.447, 0.47]
std = [0.289, 0.274, 0.278]
image = cv2.imread("/home/data/TestImg/tuchong/5.jpg")
image = ((image/255.-mean)/std).astype(np.float32)
image = cv2.resize(image,(256, 256))
input = image.transpose(2, 0, 1)
input = torch.from_numpy(input)[None]

In [ ]:
net_time = time.time()
for i in range(1000):
    model(input)
print (time.time()-net_time)

In [ ]:
torch.save(model, 'test_model.pth')

### 在私有test数据集上进行测试

In [ ]:
def trans2dbform(file,output_file):
    # 21 38 57 58 62
    with open(file, 'r') as f:
        data = f.readlines()
    f = open(output_file, 'a')
    for line in data:
        if line[0]=='#':
            f.write(line)
        else:
            line = line.split(' ')
            line = line[0]+' '+line[1]+' '+str(int(line[2])-int(line[0]))+' '+str(int(line[3])-int(line[1]))+' '+line[49]+' '+line[50]+' 0 '+line[83]+' '+line[84]+' 0 '+line[121]+' '+line[122]+' 0 '+line[123]+' '+line[124]+' 0 '+line[131]+' '+line[132]+' 0 1\n'
            f.write(line)

In [151]:
model = dbface(has_landmark=True, wide=64, has_ext=True, upmode="UCBA")
model.load('./model/model_file/dbface_nearsmall_rubust_selfdata5.pth')
model.eval()
model.cuda()
mean = [0.408, 0.447, 0.47]
std = [0.289, 0.274, 0.278]
label_file = '/home/data/Datasets/SD/self_test/label.txt'
images_dir = '/home/data/Datasets/SD/self_test/images/'
giou_loss = GIoULoss()

In [152]:
def eval_reg(gt):
    imgfile, objs = gt
    img = cv2.imread(imgfile)
    H, W, _ = img.shape
    mhw = max(H, W)
    scale = mhw / 256
    new_img = np.zeros((mhw, mhw, 3), np.float32)
    reg_tlrb = np.zeros((1 * 4, 64, 64), np.float32)
    reg_mask = np.zeros((1, 64, 64), np.float32)
    new_img[:H, :W, :] = img
    img = cv2.resize(new_img, (256,256))
    
    for obj in objs:
        obj.x = obj.x / scale
        obj.y = obj.y / scale
        obj.r = obj.r / scale
        obj.b = obj.b / scale
        cx, cy = obj.safe_scale_center(1 / 4, 64, 64)
        reg_box = np.array(obj.box) / 4
        reg_tlrb[:, cy, cx] = reg_box
        reg_mask[0, cy, cx] = 1
        
    reg_tlrb = torch.tensor(reg_tlrb).unsqueeze(0).cuda()
    reg_mask = torch.tensor(reg_mask).unsqueeze(0).cuda()
    img = ((img/255.-mean)/std).astype(np.float32)
    img = cv2.resize(img,(256, 256))
    img = img.transpose(2, 0, 1)
    img = torch.from_numpy(img)[None].cuda()
    hm, pred_tlrb, _  = model(img)
    reg_loss = giou_loss(pred_tlrb, reg_tlrb, reg_mask)
    return reg_loss
#     return reg_tlrb, pred_tlrb

In [153]:
test_gt = preprocess.load_webface(label_file, images_dir)
reg_loss = []
for i in tqdm(range(len(test_gt))):
    reg_loss.append(eval_reg(test_gt[i]).data.cpu().numpy())
reg_loss

100%|██████████| 710/710 [00:19<00:00, 37.16it/s]


[array(0.1209228, dtype=float32),
 array(0.30712637, dtype=float32),
 array(0.11098802, dtype=float32),
 array(0.20024818, dtype=float32),
 array(0.09619784, dtype=float32),
 array(0.23667486, dtype=float32),
 array(0.06635213, dtype=float32),
 array(0.20699584, dtype=float32),
 array(0.18484418, dtype=float32),
 array(0.22682822, dtype=float32),
 array(0.12497613, dtype=float32),
 array(0.06922859, dtype=float32),
 array(0.2348684, dtype=float32),
 array(0.09749871, dtype=float32),
 array(0.1265949, dtype=float32),
 array(0.1388011, dtype=float32),
 array(0.09931743, dtype=float32),
 array(0.26148504, dtype=float32),
 array(0.31053397, dtype=float32),
 array(0.20017973, dtype=float32),
 array(0.1480341, dtype=float32),
 array(0.25000584, dtype=float32),
 array(0.14936692, dtype=float32),
 array(0.11318392, dtype=float32),
 array(0.10575491, dtype=float32),
 array(0.15297627, dtype=float32),
 array(0.10301048, dtype=float32),
 array(0.11636984, dtype=float32),
 array(0.13246548, dtype=

In [155]:
1-np.array(reg_loss).mean()

0.8311252146959305